In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve

In [ ]:
df = pd.read_csv('californiabin.csv')
df.head()

In [ ]:
data = df.values
data.shape

In [ ]:
attributes = data[:, :-1]
targets = data[:, [-1]]

In [ ]:
train_attributes, test_attributes, train_targets, test_targets = train_test_split(attributes, targets, test_size=0.2, random_state=42)

In [ ]:
kfold = KFold(n_splits= 10, shuffle= True, random_state= 42)

In [ ]:
svc = SVC(kernel='rbf')

In [ ]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('svc', svc)])

In [ ]:
gs = GridSearchCV(estimator= pipe, param_grid= hyperparameters, cv= 5, n_jobs=-1)

In [ ]:
gs.fit(train_attributes, train_targets)

In [ ]:
gs.score(test_attributes, test_targets)

In [ ]:
cv_results = pd.DataFrame(gs.cv_results_)
cv_results.head()

In [ ]:
best_model = gs.best_estimator_
best_model.fit(train_attributes, train_targets)


prediction = best_model.predict(test_attributes)

In [ ]:
roc = roc_curve(test_targets, prediction)
roc

In [ ]:
prec = precision_score(test_targets, prediction)
prec

In [ ]:
f1 = f1_score(test_targets, prediction)
f1

In [ ]:
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

results = [
    [None, None, None, None],
    [None, None, None, None]
]